<a href="https://colab.research.google.com/github/a150730/Mustakim/blob/main/1st_dive_in_(_Twitter_sentiment_Analysis_Malay_Language).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install malaya
!pip install malaya-gpu
!pip install tweepy

In [2]:
%%time
import pandas as pd 
import tweepy
import malaya
from textblob import TextBlob
import re
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pickle

/usr/local/lib/python3.7/dist-packages/malaya/function/__init__.py:32: UserWarning: Cannot import beam_search_ops not available for Tensorflow 2, `deep_model` for stemmer will not available to use.
  'Cannot import beam_search_ops not available for Tensorflow 2, `deep_model` for stemmer will not available to use.'


CPU times: user 4.63 s, sys: 671 ms, total: 5.3 s
Wall time: 5.24 s


# **Accessing twitter API -using tweepy**

In [3]:
consumer_key ="kjiUiDnn3VgEYpcQirYUWfu8f"
consumer_secret ="kYUdb0ebDtC6D92DDYvEqKShcKVqjgZ3048vyzxNprTua0MYUW"
access_token ="2301609108-l6W2GoHZUHK3ELBWSb4D4fN0J7FIbNe0BLkJQHV"
access_token_secret ="6h64rvQXSvltg9pZBUL3iuVHFKx48q0yvxQtxpWUAMY8C"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit = True)

* extract tweets

In [4]:
%%time
query='#KerajaanGagal -filter:retweets' # this query will exclude retweets
post = api.search(q=query, count=10)

CPU times: user 25.4 ms, sys: 0 ns, total: 25.4 ms
Wall time: 384 ms


In [5]:
i=1
for tweet in post[0:10]:
  print (str(i)+')' + tweet.text +'\n')
  i+=1

1)Tone policing is fascism masked as politeness @KKMPutrajaya people are angry, people are suicidal, but sure, bAhAsA… https://t.co/f7eKC7K4mP

2)Patut lah berduyun-duyun orang nak jadi ahli PPBM, kalah masa jadi calon pun boleh dilantik menteri atau timbalan menteri. #KerajaanGagal

3)@Marina_Ibrahims Tengok rambut pon da tau level akal dia da tak boleh pakai. Kalau tu bapak aku, da lama aku nasiha… https://t.co/ILMqz5tvUO

4)It has been months and the numbers are still high. No aid, no support, no relief. Just stolen futures and wealth di… https://t.co/GWcapKNZVY

5)@eqwanroslan @MaisarraAnuar #KerajaanGagal

6)Tapi ekonomi pun merudum
#KerajaanGagal 
#KerajaanGagal 
#KerajaanGagal https://t.co/uq0UWCHPn7

7)dalam lagaknya tolak kebebaran, india akhirnya tumpas. 

this could be us.

perang belum selesai. #KerajaanGagal da… https://t.co/5nswPM9qug

8)@SyedSaddiq I know what’s the problem. They do not know what DOUBLE STANDARD means. That’s why they keep doing it. #KerajaanGagal

9)#Ta

In [6]:
#convert data into dataframe
df = pd.DataFrame([tweet.text for tweet in post], columns =['Raw'])
df.info() #just checking the datafranme info and data typw

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Raw     10 non-null     object
dtypes: object(1)
memory usage: 208.0+ bytes


In [7]:
df.style #view our dataframe

,Raw
0,"Tone policing is fascism masked as politeness @KKMPutrajaya people are angry, people are suicidal, but sure, bAhAsA… https://t.co/f7eKC7K4mP"
1,"Patut lah berduyun-duyun orang nak jadi ahli PPBM, kalah masa jadi calon pun boleh dilantik menteri atau timbalan menteri. #KerajaanGagal"
2,"@Marina_Ibrahims Tengok rambut pon da tau level akal dia da tak boleh pakai. Kalau tu bapak aku, da lama aku nasiha… https://t.co/ILMqz5tvUO"
3,"It has been months and the numbers are still high. No aid, no support, no relief. Just stolen futures and wealth di… https://t.co/GWcapKNZVY"
4,@eqwanroslan @MaisarraAnuar #KerajaanGagal
5,Tapi ekonomi pun merudum #KerajaanGagal #KerajaanGagal #KerajaanGagal https://t.co/uq0UWCHPn7
6,"dalam lagaknya tolak kebebaran, india akhirnya tumpas. this could be us. perang belum selesai. #KerajaanGagal da… https://t.co/5nswPM9qug"
7,@SyedSaddiq I know what’s the problem. They do not know what DOUBLE STANDARD means. That’s why they keep doing it. #KerajaanGagal
8,#TamatDarurat #KerajaanGagal #TolakPNipu https://t.co/vQJD33OZXs
9,Better? You’re havin’ a laugh eh @KDNPUTRAJAYA ? #KerajaanGagal #TamatDarurat https://t.co/v2ALZB1On2


## **Data PreProcessing**
* 1st with Malaya
*  Second with re

In [8]:
#initialize prepprocessing model
%%time
mPP = malaya.preprocessing.preprocessing(translate_english_to_bm=True, lowercase=True)

CPU times: user 212 ms, sys: 41.1 ms, total: 253 ms
Wall time: 255 ms


/usr/local/lib/python3.7/dist-packages/malaya/preprocessing.py:84: FutureWarning: Possible nested set at position 42
  k.lower(): re.compile(_expressions[k]) for k, v in _expressions.items()
/usr/local/lib/python3.7/dist-packages/malaya/preprocessing.py:259: FutureWarning: Possible nested set at position 2289
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [9]:
def preProcessMalaya(text):
  return ' '.join(mPP.process(text))

In [10]:
df['cleanData-M'] = df['Raw'].astype(str).apply(preProcessMalaya)

In [11]:
pd.options.display.max_colwidth =70 #extend the coulumn width
pd.options.display.max_rows = 10
df.style

,Raw,cleanData-M
0,"Tone policing is fascism masked as politeness @KKMPutrajaya people are angry, people are suicidal, but sure, bAhAsA… https://t.co/f7eKC7K4mP","tone policing adalah fasisme bertopeng sebagai kesopanan orang adalah marah , orang adalah bunuh diri , tetapi pasti , bahasa …"
1,"Patut lah berduyun-duyun orang nak jadi ahli PPBM, kalah masa jadi calon pun boleh dilantik menteri atau timbalan menteri. #KerajaanGagal","patut lah berduyun-duyun orang nak jadi ahli ppbm , kalah masa jadi calon pun boleh dilantik menteri atau timbalan menteri . kerajaangagal"
2,"@Marina_Ibrahims Tengok rambut pon da tau level akal dia da tak boleh pakai. Kalau tu bapak aku, da lama aku nasiha… https://t.co/ILMqz5tvUO","tengok rambut pun dah tahu tahap akal dia dah tidak boleh pakai . kalau itu bapak aku , dah lama aku nasiha …"
3,"It has been months and the numbers are still high. No aid, no support, no relief. Just stolen futures and wealth di… https://t.co/GWcapKNZVY","ia has telah months dan yang numbers adalah masih ada high . tidak bantuan , tidak sokongan , tidak pelepasan . hanya dicuri futures dan kekayaan di …"
4,@eqwanroslan @MaisarraAnuar #KerajaanGagal,kerajaangagal
5,Tapi ekonomi pun merudum #KerajaanGagal #KerajaanGagal #KerajaanGagal https://t.co/uq0UWCHPn7,tetapi ekonomi pun merudum kerajaangagal kerajaangagal kerajaangagal
6,"dalam lagaknya tolak kebebaran, india akhirnya tumpas. this could be us. perang belum selesai. #KerajaanGagal da… https://t.co/5nswPM9qug","dalam lagaknya tolak kebebaran , india akhirnya tumpas . this boleh jadi kami . perang belum selesai . kerajaangagal dah …"
7,@SyedSaddiq I know what’s the problem. They do not know what DOUBLE STANDARD means. That’s why they keep doing it. #KerajaanGagal,saya tahu apa ' s yang masalah . they do tidak tahu apa double standard means . itu ' s kenapa they teruskan melakukan ia . kerajaangagal
8,#TamatDarurat #KerajaanGagal #TolakPNipu https://t.co/vQJD33OZXs,tamatdarurat kerajaangagal tolakpnipu
9,Better? You’re havin’ a laugh eh @KDNPUTRAJAYA ? #KerajaanGagal #TamatDarurat https://t.co/v2ALZB1On2,lebih baik ? awak adalah havin ' a ketawa eh ? kerajaangagal tamatdarurat


* install googletrans API for language translation part:
   *  my plan is for **re** preprocessing there is no langugae translation module , so I am going to use googletrans API ver 3.1.0a0 since latest ones got some error.

In [12]:
!pip install googletrans==4.0.0-rc1

In [13]:
import googletrans as gt

In [14]:
print(gt.LANGUAGES)#check available languages

{'af': 'afrikaans', 'sq': 'albanian', 'am': 'amharic', 'ar': 'arabic', 'hy': 'armenian', 'az': 'azerbaijani', 'eu': 'basque', 'be': 'belarusian', 'bn': 'bengali', 'bs': 'bosnian', 'bg': 'bulgarian', 'ca': 'catalan', 'ceb': 'cebuano', 'ny': 'chichewa', 'zh-cn': 'chinese (simplified)', 'zh-tw': 'chinese (traditional)', 'co': 'corsican', 'hr': 'croatian', 'cs': 'czech', 'da': 'danish', 'nl': 'dutch', 'en': 'english', 'eo': 'esperanto', 'et': 'estonian', 'tl': 'filipino', 'fi': 'finnish', 'fr': 'french', 'fy': 'frisian', 'gl': 'galician', 'ka': 'georgian', 'de': 'german', 'el': 'greek', 'gu': 'gujarati', 'ht': 'haitian creole', 'ha': 'hausa', 'haw': 'hawaiian', 'iw': 'hebrew', 'he': 'hebrew', 'hi': 'hindi', 'hmn': 'hmong', 'hu': 'hungarian', 'is': 'icelandic', 'ig': 'igbo', 'id': 'indonesian', 'ga': 'irish', 'it': 'italian', 'ja': 'japanese', 'jw': 'javanese', 'kn': 'kannada', 'kk': 'kazakh', 'km': 'khmer', 'ko': 'korean', 'ku': 'kurdish (kurmanji)', 'ky': 'kyrgyz', 'lo': 'lao', 'la': 'lat

In [15]:
from googletrans import Translator  

In [16]:
translator = Translator()

In [17]:
text1 ='good morning everybody'
text2 ='Apa khabar awak?'
text3 ='안녕하세요'
text4 = " Apa khabar awak?.Im fine terima kasih"

In [18]:
print(translator.detect(text1))
print(translator.detect(text2))
print(translator.detect(text3))
print(translator.detect(text4))
print(translator.translate('안녕하세요.'))
print(translator.translate(text1, dest='ms'))


Detected(lang=en, confidence=None)
Detected(lang=ms, confidence=None)
Detected(lang=ko, confidence=None)
Detected(lang=ms, confidence=None)
Translated(src=ko, dest=en, text=Good morning., pronunciation=None, extra_data="{'confiden...")
Translated(src=en, dest=ms, text=selamat Pagi semua, pronunciation=None, extra_data="{'confiden...")


* 2 pieces of code above just to try to detect and translate language 

In [19]:
 #Translate english word to bm
    # the steps as as follow:
    # 1)split the string into words
    # 2) detect words language 
    #     *if en --> covert
    #     * else -->skip
    # 3) append the string
    
def trans_En_to_Bm(text):
  sent2Word = text.split() #step1:splitting/ tekenizing the sentence
  convText=[]
  
  for word in sent2Word:
    lang = translator.detect(word)# step2)-1 :detect language
    if (lang.lang =='en'):
      word = translator.translate(word, dest='ms') #step2)-2 :translate english to bahasa melayu
      convText.append(word.text)
    else:
      convText.append(word)

  text = ' '.join(convText)

  return text
  convText.clear()

In [20]:
#removing emoji/emoticon
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [21]:
with open('/content/drive/MyDrive/Colab Notebooks/Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

def convert_emojis_to_word(text):
    for emoj in Emoji_Dict:
        text = re.sub(r'('+emoj+')', "_".join(Emoji_Dict[emoj].replace(",","").replace(":","").split()), text)
    return text

In [22]:
#define class for re preperocessing

def preProcessMus(text):
  
  text = text.lower()

  #1st step <---- remove emoji and emoticon
  #text = remove_emoji(text)
  #covert emoji/emoticon to word
  text = convert_emojis_to_word(text)
  text = re.sub(r'https?:\/\/\S+', ' ', text)#remove hyperlink


  #2nd step<--- remove special character and number
  # after ....re.sub(... the single 'r' stands for raw string. 
  #so it will ignore all the escape character python has. 
  #as for example '\n' which refer as new line. it will ignore
  text=re.sub(r'/\b\S\s\b\n/', '', text)
  text= re.sub(r'@[A-Z_a-z_0-9]+', '', text)#remove @mentions or user with '_'  
  text= re.sub(r'@[A-Z0-9a-z0-9]+', '', text)#remove @mentions or user that dont have '_'
  text = re.sub(r'#[A-Za-z0-9_A-Za-z0-9]+', '',text)#remove #hashtag 
  text= re.sub(r'\n', '', text)#remove \n
  text=re.sub(r'[-()\"''#!@$%^&*{}?.,:]',' ',text)#remove speial character and punctuation
  text =' '.join(text.split('_')) # split word that merge by underscore '_'

  #4th step<--- translate english word to bahasa
  text = trans_En_to_Bm(text) # have to commented out this because googletrans cant handle huge data
  text=re.sub(r'[-()\"''#!@$%^&*{}?.,:]',' ',text)#remove speial character and punctuation

  #3rd step<--- first lowercase the text
  text = text.lower()

  return text

In [23]:
text5 = '@Marina_Ibrahims Kebabian PN makin terserlah dan muhyiddin kimak\n#KerajaanGagal'
text6 = '@kuasasiswa you made my day!👏😂\nso far the BEST troll ever!\nso much effort you put in👍\n#Keraja...'
text7 = '@Mar5s6dgs0d9 🐷 \nmaroni kimak\n#Kerajaan_Gagal'

print(preProcessMus(text5))
print(preProcessMus(text6))
print(preProcessMus(text7))

kebabian pn  makin  terserlah dan muhyiddin kimak
anda dibuat my hari bertepuk tangan handsface  dengan air mata of  joyso jauh the best  troll  pernah so  banyak usaha anda letak inthumbs  up
babi muka maroni kimak


In [24]:
%%time
df['cleanData-MUS'] = df['Raw'].astype(str).apply(preProcessMus)

CPU times: user 3.36 s, sys: 171 ms, total: 3.53 s
Wall time: 15.7 s


In [25]:
df.style

,Raw,cleanData-M,cleanData-MUS
0,"Tone policing is fascism masked as politeness @KKMPutrajaya people are angry, people are suicidal, but sure, bAhAsA… https://t.co/f7eKC7K4mP","tone policing adalah fasisme bertopeng sebagai kesopanan orang adalah marah , orang adalah bunuh diri , tetapi pasti , bahasa …",nada polis adalah fasisme bertopeng kesopanan orang ramai adalah marah orang ramai adalah suicidal tetapi pasti bahasa…
1,"Patut lah berduyun-duyun orang nak jadi ahli PPBM, kalah masa jadi calon pun boleh dilantik menteri atau timbalan menteri. #KerajaanGagal","patut lah berduyun-duyun orang nak jadi ahli ppbm , kalah masa jadi calon pun boleh dilantik menteri atau timbalan menteri . kerajaangagal",patut lah berduyun duyun orang nak jadi ahli ppbm kalah masa jadi calon pun boleh dilantik menteri atau timbalan menteri
2,"@Marina_Ibrahims Tengok rambut pon da tau level akal dia da tak boleh pakai. Kalau tu bapak aku, da lama aku nasiha… https://t.co/ILMqz5tvUO","tengok rambut pun dah tahu tahap akal dia dah tidak boleh pakai . kalau itu bapak aku , dah lama aku nasiha …",tengok rambut pon da tau tahap akal dia da tak boleh pakai kalau tu bapak aku da lama aku nasiha…
3,"It has been months and the numbers are still high. No aid, no support, no relief. Just stolen futures and wealth di… https://t.co/GWcapKNZVY","ia has telah months dan yang numbers adalah masih ada high . tidak bantuan , tidak sokongan , tidak pelepasan . hanya dicuri futures dan kekayaan di …",ia telah telah bulan dan the nombor adalah masih ada tinggi tidak bantuan tidak sokongan tidak kelegaan hanya dicuri niaga hadapan dan kekayaan di…
4,@eqwanroslan @MaisarraAnuar #KerajaanGagal,kerajaangagal,
5,Tapi ekonomi pun merudum #KerajaanGagal #KerajaanGagal #KerajaanGagal https://t.co/uq0UWCHPn7,tetapi ekonomi pun merudum kerajaangagal kerajaangagal kerajaangagal,tapi ekonomi pun merudum
6,"dalam lagaknya tolak kebebaran, india akhirnya tumpas. this could be us. perang belum selesai. #KerajaanGagal da… https://t.co/5nswPM9qug","dalam lagaknya tolak kebebaran , india akhirnya tumpas . this boleh jadi kami . perang belum selesai . kerajaangagal dah …",dalam lagaknya tolak kebebaran india akhirnya tumpas ini boleh akan kami perang belum selesai da…
7,@SyedSaddiq I know what’s the problem. They do not know what DOUBLE STANDARD means. That’s why they keep doing it. #KerajaanGagal,saya tahu apa ' s yang masalah . they do tidak tahu apa double standard means . itu ' s kenapa they teruskan melakukan ia . kerajaangagal,i tahu apa itu the masalah mereka lakukan tidak tahu apa double standard bererti itu kenapa mereka menyimpan melakukan ia
8,#TamatDarurat #KerajaanGagal #TolakPNipu https://t.co/vQJD33OZXs,tamatdarurat kerajaangagal tolakpnipu,
9,Better? You’re havin’ a laugh eh @KDNPUTRAJAYA ? #KerajaanGagal #TamatDarurat https://t.co/v2ALZB1On2,lebih baik ? awak adalah havin ' a ketawa eh ? kerajaangagal tamatdarurat,lebih baik anda havin ' a ketawa eh


In [26]:
df

,Raw,cleanData-M,cleanData-MUS
0,Tone policing is fascism masked as politeness @KKMPutrajaya people...,tone policing adalah fasisme bertopeng sebagai kesopanan <user> or...,nada polis adalah fasisme bertopeng kesopanan orang ramai adalah...
1,"Patut lah berduyun-duyun orang nak jadi ahli PPBM, kalah masa jadi...",patut lah berduyun-duyun orang nak jadi ahli <allcaps> ppbm </allc...,patut lah berduyun duyun orang nak jadi ahli ppbm kalah masa jadi ...
2,@Marina_Ibrahims Tengok rambut pon da tau level akal dia da tak bo...,<user> tengok rambut pun dah tahu tahap akal dia dah tidak boleh p...,tengok rambut pon da tau tahap akal dia da tak boleh pakai kalau t...
3,"It has been months and the numbers are still high. No aid, no supp...",ia has telah months dan yang numbers adalah masih ada high . tidak...,ia telah telah bulan dan the nombor adalah masih ada tinggi tidak ...
4,@eqwanroslan @MaisarraAnuar #KerajaanGagal,<user> <user> <hashtag> kerajaangagal </hashtag>,
5,Tapi ekonomi pun merudum\n#KerajaanGagal \n#KerajaanGagal \n#Keraj...,tetapi ekonomi pun merudum <hashtag> kerajaangagal </hashtag> <has...,tapi ekonomi pun merudum
6,"dalam lagaknya tolak kebebaran, india akhirnya tumpas. \n\nthis co...","dalam lagaknya tolak kebebaran , india akhirnya tumpas . this bole...",dalam lagaknya tolak kebebaran india akhirnya tumpas ini boleh aka...
7,@SyedSaddiq I know what’s the problem. They do not know what DOUBL...,<user> saya tahu apa ' s yang masalah . they do tidak tahu apa <al...,i tahu apa itu the masalah mereka lakukan tidak tahu apa double ...
8,#TamatDarurat #KerajaanGagal #TolakPNipu https://t.co/vQJD33OZXs,<hashtag> tamatdarurat </hashtag> <hashtag> kerajaangagal </hashta...,
9,Better? You’re havin’ a laugh eh @KDNPUTRAJAYA ?\n\n#KerajaanGagal...,lebih baik ? awak adalah havin ' a ketawa eh <user> ? <hashtag> ke...,lebih baik anda havin ' a ketawa eh
